<a href="https://colab.research.google.com/github/jorgewellison/classifiers-tcc/blob/main/Naive-bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Configurações iniciais**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Importação das bibliotecas**

In [60]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import seaborn as sns
from numpy import nper
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import make_scorer, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from yellowbrick.classifier import ConfusionMatrix
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold, cross_validate
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder




**Importando previsores e classe (tratados no pré-processamento)**

previsores -> variáveis independentes - X

---

classe -> variável dependente - y

In [3]:
previsores = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid-idhm/previsores_.csv") 
classe_inc = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid-idhm/classe_inc.csv") 
classe_mort = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid-idhm/classe_mort.csv")

#**Criação, treinamento dos modelos e visualização de atributos mais importantes**

##**Incidência**##

**Treinamento**

---



In [42]:
naive_bayes_inc = MultinomialNB()
naive_bayes_inc.fit(previsores, classe_inc.values.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

**Obtenção dos atributos mais importantes**

---

In [44]:
count_vect = CountVectorizer() 
final_counts = count_vect.fit_transform(previsores)

In [43]:
previsores_prob_sorted = naive_bayes_inc.feature_log_prob_[0, :].argsort()

In [79]:
nb_importances = (np.take(count_vect.get_feature_names(), previsores_prob_sorted[-10:]))
print(nb_importances)

['t_banagua' 't_crifundin_todos' 'p_med' 't_ocupdesloc_1'
 't_nestuda_ntrab_mmeio' 't_mulchefefif014' 't_sluz' 't_super25m'
 't_rmaxidoso' 'trabsc']


**Representação de atributos mais importantes em forma de gráfico**

---




In [ ]:
#Gerar gráfico com os detalhes de cada atributo

##**Mortalidade**##

**Treinamento**

---



In [75]:
naive_bayes_mort = MultinomialNB()
naive_bayes_mort.fit(previsores, classe_mort.values.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

**Obtenção dos atributos mais importantes**

---

In [81]:
previsores_prob_sorted2 = naive_bayes_mort.feature_log_prob_[0, :].argsort()

In [83]:
nb_importances2 = (np.take(count_vect.get_feature_names(), previsores_prob_sorted2[-10:]))
print(nb_importances2)

['t_banagua' 't_crifundin_todos' 'p_med' 't_ocupdesloc_1'
 't_nestuda_ntrab_mmeio' 't_mulchefefif014' 't_sluz' 't_super25m'
 't_rmaxidoso' 'trabsc']


**Representação de atributos mais importantes em forma de gráfico**

---




In [ ]:
#Gerar gráfico com os detalhes de cada atributo

#**Testes com validação cruzada**

**Criação do KFold com 10 iterações e Shuffle para embaralhar as linhas, desagrupando os municipios de seus estados**

In [84]:
cv = KFold(n_splits=10, random_state=0, shuffle=True)

In [87]:
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average = 'macro'),
           'recall' : make_scorer(recall_score, average = 'macro'), 
           'f1_score' : make_scorer(f1_score, average = 'macro')}

##Testes de incidência##

In [89]:
#a função cross_validate recebe por parâmetro o modelo
#todos os dados para treino, os dados de classes, a quandidade de folds e o metodo de avaliação

scores = cross_validate(naive_bayes_inc, 
                        previsores, 
                        classe_inc.values.ravel(),
                        cv=cv,
                        scoring=scoring,
                        return_train_score=False)

###Avaliação dos testes###

In [90]:
scores

{'fit_time': array([0.01179314, 0.00766087, 0.00730228, 0.00756121, 0.00753617,
        0.00775743, 0.00754976, 0.00774789, 0.0076611 , 0.00756001]),
 'score_time': array([0.00954247, 0.00507879, 0.00580907, 0.00514627, 0.00503874,
        0.00519609, 0.00507092, 0.00527263, 0.00514269, 0.0050106 ]),
 'test_accuracy': array([0.59066427, 0.60502693, 0.57809695, 0.5978456 , 0.56373429,
        0.55755396, 0.60251799, 0.60791367, 0.60431655, 0.60071942]),
 'test_f1_score': array([0.44133038, 0.42772147, 0.4282142 , 0.45240895, 0.42126178,
        0.42050039, 0.44844227, 0.44328833, 0.43397368, 0.46762602]),
 'test_precision': array([0.51205598, 0.47060851, 0.48853402, 0.53102419, 0.49317903,
        0.54176951, 0.49243954, 0.49762556, 0.46166244, 0.53424338]),
 'test_recall': array([0.46159931, 0.44458451, 0.45182743, 0.46807236, 0.43867652,
        0.45496858, 0.4604457 , 0.45155917, 0.44544267, 0.48329977])}

##Testes de mortalidade##

In [91]:
#a função cross_validate recebe por parâmetro o modelo
#todos os dados para treino, os dados de classes, a quandidade de folds e o metodo de avaliação

scores2 = cross_validate(naive_bayes_mort, 
                         previsores, 
                         classe_mort.values.ravel(),
                         scoring=scoring,
                         cv=cv,
                         return_train_score=False)

###Avaliação dos testes###

In [92]:
scores2

{'fit_time': array([0.01694536, 0.00794125, 0.0083096 , 0.00802875, 0.00753522,
        0.00782704, 0.00745797, 0.00785589, 0.0074048 , 0.00732374]),
 'score_time': array([0.00474787, 0.00526714, 0.00568962, 0.00518346, 0.00649285,
        0.00529242, 0.00508761, 0.0052712 , 0.00514698, 0.00521064]),
 'test_accuracy': array([0.57091562, 0.57450628, 0.53141831, 0.55655296, 0.5475763 ,
        0.54496403, 0.55395683, 0.55935252, 0.55395683, 0.53956835]),
 'test_f1_score': array([0.5058252 , 0.50049398, 0.46307774, 0.47520917, 0.47110427,
        0.46192784, 0.48699106, 0.48432989, 0.50397171, 0.48119929]),
 'test_precision': array([0.56027217, 0.57107407, 0.54084152, 0.54078376, 0.562421  ,
        0.57451637, 0.57204004, 0.55970209, 0.54858413, 0.55884899]),
 'test_recall': array([0.52507446, 0.52356513, 0.49268618, 0.48565487, 0.498414  ,
        0.49100331, 0.51440856, 0.50836296, 0.52712585, 0.50991392])}

# **Predição e Matriz de confusão**

### Predição e Matriz da incidência

In [94]:
predictions = cross_val_predict(naive_bayes_inc, previsores, classe_inc.values.ravel(), cv=cv)
confusao = confusion_matrix(classe_inc, predictions)
confusao

array([[1062,  177, 1395],
       [  51,   38,  342],
       [ 145,  167, 2188]])

### Predição e Matriz da mortalidade

In [97]:
predictions2 = cross_val_predict(naive_bayes_mort, previsores, classe_mort.values.ravel(), cv=cv)
confusao2 = confusion_matrix(classe_mort, predictions2)
confusao2

array([[1903,  159,  100],
       [ 537,  226,   66],
       [1289,  335,  950]])

# **Métricas de avaliação**

**Predição de Incidência**

In [99]:
metrics.accuracy_score(classe_inc, predictions)

0.5908355795148248

In [100]:
metrics.precision_score(classe_inc, predictions, average=None)

array([0.84419714, 0.09947644, 0.55745223])

In [101]:
metrics.recall_score(classe_inc, predictions, average=None)

array([0.40318907, 0.08816705, 0.8752    ])

In [102]:
metrics.f1_score(classe_inc, predictions, average=None)

array([0.54573484, 0.09348093, 0.68108949])

**Predição de Mortalidade**

In [103]:
metrics.accuracy_score(classe_mort, predictions2)

0.5532794249775382

In [104]:
metrics.precision_score(classe_mort, predictions2, average=None)

array([0.51032448, 0.31388889, 0.85125448])

In [105]:
metrics.recall_score(classe_mort, predictions2, average=None)

array([0.88020352, 0.27261761, 0.36907537])

In [106]:
metrics.f1_score(classe_mort, predictions2, average=None)

array([0.64607028, 0.29180116, 0.51490515])